In [23]:
# Classical Tail Assignment (PuLP) - Clean Notebook
# This notebook builds an integer program for the Tail Assignment Problem (TAP)
# using only feasible (flight, tail) decision variables (fleet+capacity+listed).
# It solves with PuLP (CBC), validates, and writes the assignment to JSON.
#
# Save location: /mnt/data/classical_fixed.ipynb (this notebook)
# Output assignment: /mnt/data/classical_assignment_fixed.json
#
# Run all cells in order.

# If needed, uncomment to install pulp in your environment:
# !pip install pulp

import json
from collections import defaultdict
import pulp


In [24]:
# --- Load problem data ---
DATA_PATH = "problem_data_420f_32t.json"  # change to _2x/_8x/_12x versions as needed
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

flights = {f['id']: f for f in data['flights']}
tails = {t['id']: t for t in data['tails']}
costs = data['costs']
max_costs = data['max_costs']
impossible_pairs = set(tuple(p) for p in data.get('impossible_pairings', []))
tuning_params = data.get('tuning_parameters', {})

print(f"Loaded {len(flights)} flights and {len(tails)} tails.")


Loaded 420 flights and 32 tails.


In [25]:
# --- Quick diagnostics ---
valid_counts = {}
no_valid = []
for f_id, f in flights.items():
    valid = [t_id for t_id, t in tails.items() if t['fleet']==f['fleet_req'] and t['capacity']>=f['seats_req'] and f_id in costs and t_id in costs[f_id]]
    valid_counts[f_id] = len(valid)
    if len(valid)==0:
        no_valid.append(f_id)
print('Min/Max valid tails per flight:', min(valid_counts.values()), max(valid_counts.values()))
print('Flights with zero valid tails (should be none):', no_valid[:10])
print('Impossible pair count:', len(impossible_pairs))


Min/Max valid tails per flight: 2 12
Flights with zero valid tails (should be none): []
Impossible pair count: 2525


In [26]:
# --- Build PuLP model with only valid variables ---
prob = pulp.LpProblem('Tail_Assignment', pulp.LpMinimize)

# Decision variables: x[(f,t)] only when fleet & capacity & listed in costs
x = {}
for f_id, f in flights.items():
    for t_id, t in tails.items():
        fleet_ok = (t['fleet'] == f['fleet_req'])
        seats_ok = (t['capacity'] >= f['seats_req'])
        listed = (f_id in costs and t_id in costs[f_id])
        if fleet_ok and seats_ok and listed:
            x[(f_id, t_id)] = pulp.LpVariable(f"x_{f_id}_{t_id}", cat='Binary')

print('Total valid (f,t) variables:', len(x))
# Sanity check: ensure each flight has at least one option
for f_id in flights:
    if not any(k for k in x.keys() if k[0]==f_id):
        raise RuntimeError(f"No valid tail options for flight {f_id}")


Total valid (f,t) variables: 3598


In [27]:
# --- Constraints ---
# 1) Each flight assigned exactly one tail (one-hot over available vars)
for f_id in flights:
    vars_for_f = [x[(f_id,t)] for (f_id2,t) in x.keys() if f_id2==f_id]
    prob += pulp.lpSum(vars_for_f) == 1, f"one_tail_{f_id}"

# 2) Impossible pair constraints: for each impossible pair (f1,f2) and tail t, disallow both
for (f1, f2) in impossible_pairs:
    for t_id in tails:
        if (f1, t_id) in x and (f2, t_id) in x:
            prob += x[(f1,t_id)] + x[(f2,t_id)] <= 1, f"impossible_{f1}_{f2}_{t_id}"


In [28]:
# --- Objective ---
prob += pulp.lpSum((costs[f][t] * x[(f,t)]) for (f,t) in x), 'Total_Cost'


In [29]:
# --- Solve with a time limit ---
time_limit_seconds = 120  # adjust as needed
solver = pulp.PULP_CBC_CMD(timeLimit=int(time_limit_seconds), msg=True)
status = prob.solve(solver)
print('Solver status:', pulp.LpStatus[prob.status])


Solver status: Optimal


In [30]:
# --- Extract assignment (flight -> tail) ---
classical_assignment = {}
for (f,t), var in x.items():
    val = var.value()
    if val is not None and float(val) > 0.5:
        classical_assignment[f] = t

print('Flights assigned:', len(classical_assignment), '/', len(flights))
missing = [f for f in flights if f not in classical_assignment]
if missing:
    print('Missing flights (first 20):', missing[:20])

# --- Validate fleet/capacity violations ---
violations = []
for f_id, t_id in classical_assignment.items():
    f = flights[f_id]; t = tails[t_id]
    if t['fleet'] != f['fleet_req']:
        violations.append((f_id, t_id, f"fleet mismatch {t['fleet']} != {f['fleet_req']}"))
    if t['capacity'] < f['seats_req']:
        violations.append((f_id, t_id, f"insufficient capacity required {f['seats_req']}, got {t['capacity']}"))

print('Fleet/capacity violations:', len(violations))
for v in violations[:50]:
    print(' ', v)

# --- Validate impossible-pair violations ---
tail_to_f = defaultdict(list)
for f,t in classical_assignment.items():
    tail_to_f[t].append(f)

impossible_viol = []
impset = set(tuple(p) for p in data.get('impossible_pairings', []))
for t, flist in tail_to_f.items():
    fs = sorted(flist)
    for i in range(len(fs)):
        for j in range(i+1, len(fs)):
            a, b = fs[i], fs[j]
            if (a,b) in impset or (b,a) in impset:
                impossible_viol.append((t, a, b))
print('Impossible-pair violations:', len(impossible_viol))
for v in impossible_viol[:50]:
    print(' ', v)

# --- Normalized cost ---
def total_normalized_cost(assign):
    total = 0.0
    for f_id, t_id in assign.items():
        c = costs.get(f_id, {}).get(t_id, None)
        if c is None:
            return float('inf')
        total += c / max_costs[f_id]
    return total

norm_cost = total_normalized_cost(classical_assignment) if len(classical_assignment)==len(flights) else float('inf')
print('Total normalized cost (or inf if infeasible/missing):', norm_cost)

# --- Save assignment ---
OUT_JSON = 'classical_assignment_fixed.json'
with open(OUT_JSON, 'w') as fh:
    json.dump(classical_assignment, fh, indent=2)
print('Saved assignment to', OUT_JSON)


Flights assigned: 420 / 420
Fleet/capacity violations: 0
Impossible-pair violations: 0
Total normalized cost (or inf if infeasible/missing): 93.37376973622172
Saved assignment to classical_assignment_fixed.json


In [31]:
# --- Also calculate and print non-normalized (raw) total cost ---
raw_cost = 0.0
for f_id, t_id in classical_assignment.items():
    if f_id in costs and t_id in costs[f_id]:
        raw_cost += costs[f_id][t_id]
    else:
        print(f"⚠️ Missing cost for pair ({f_id}, {t_id}) — skipped")

print("Total raw (non-normalized) cost:", raw_cost)
print("Total normalized cost (or inf if infeasible/missing):", norm_cost)


Total raw (non-normalized) cost: 56135.200000000084
Total normalized cost (or inf if infeasible/missing): 93.37376973622172


In [32]:
# --- (Optional) Compare with SQA assignment if present ---
import os
sqa_path = 'sqa_assignment.json'
if os.path.exists(sqa_path):
    with open(sqa_path, 'r') as f:
        sqa = json.load(f)
    print('\nSQA assignment loaded: comparing normalized costs and feasibility...')
    print('SQA flights assigned:', len(sqa), '/', len(flights))
    print('SQA normalized cost:', total_normalized_cost(sqa))
else:
    print('\nNo SQA assignment found at', sqa_path)



SQA assignment loaded: comparing normalized costs and feasibility...
SQA flights assigned: 360 / 420
SQA normalized cost: inf
